In [1]:
# Data exploration
import pandas as pd

# Numerical
import numpy as np

In [18]:
#import the functions from their corresponding files
from NYC_GetCleaned_PrecinctData import getPrecinctData
from data_utility import filterData,sortValue,getCount

In [4]:
#Get cleaned data from NYC_GetCleaned_HistoricData
crimes_original = getPrecinctData()
#crimes_original.head()

,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,OFNS_DESC,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,Latitude,Longitude,Lat_Lon,Shape_Area,Shape_Leng,Precinct,the_geom
0,3/30/2014,20:40:00,3/30/2014,20:50:00,3/30/2014,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE,INTENT TO",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,RESIDENCE - APT. HOUSE,40.836162,-73.915250,"(40.836161968, -73.915249813)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
1,4/1/2014,15:00:00,4/1/2014,15:05:00,4/1/2014,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,STREET,40.825000,-73.916615,"(40.825000207, -73.916615377)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
2,4/6/2014,14:45:00,4/6/2014,14:45:00,4/6/2014,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,STREET,40.830013,-73.918112,"(40.830013132, -73.918112312)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
3,5/6/2014,14:00:00,5/6/2014,14:05:00,5/6/2014,GRAND LARCENY OF MOTOR VEHICLE,"LARCENY,GRAND OF AUTO",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,STREET,40.828608,-73.921742,"(40.828607613, -73.92174192)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
4,3/6/2014,23:42:00,3/6/2014,23:42:00,3/7/2014,POSSESSION OF STOLEN PROPERTY,"STOLEN PROPERTY 2,1,POSSESSION",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,STREET,40.838752,-73.913758,"(40.83875187, -73.913757556)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...


In [5]:
crimes_original.shape

(5835784, 20)

In [6]:
"""Create a datetime index of times that crimes were reported to have been committed"""

dfCopy = crimes_original.copy()
def eliminate_dates(x):
    if x[2] > '2050':
        x = None
    elif x[2] < '2010':
        x = None
    else: 
        aa= '/'.join(x)
        return (aa)

#get dummy columns for crime categories
#dfCopy = dfCopy.join(dfCopy['LAW_CAT_CD'].str.get_dummies())

#dfCopy = dfCopy.join(dfCopy['BORO_NM'].str.get_dummies())

#Create index with DateTime
dfCopy['CMPLNT_FR_DT'] = dfCopy['CMPLNT_FR_DT'].str.split("/")
dfCopy['CMPLNT_FR_DT'] = dfCopy['CMPLNT_FR_DT'].apply(lambda x: eliminate_dates(x))

#Combing date and time columns
dfCopy['StartTime'] = dfCopy['CMPLNT_FR_DT'] +' '+dfCopy['CMPLNT_FR_TM']
#dfCopy['StartTime'] = dfCopy['CMPLNT_FR_TM']
dfCopy['StartTime'] = pd.to_datetime(dfCopy['StartTime'])


#set full date as index
dfCopy.set_index('StartTime', inplace=True)        

dfCopy.head()


,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,OFNS_DESC,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,Latitude,Longitude,Lat_Lon,Shape_Area,Shape_Leng,Precinct,the_geom
StartTime,,,,,,,,,,,,,,,,,,,,
2014-03-30 20:40:00,3/30/2014,20:40:00,3/30/2014,20:50:00,3/30/2014,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE,INTENT TO",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,RESIDENCE - APT. HOUSE,40.836162,-73.915250,"(40.836161968, -73.915249813)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
2014-04-01 15:00:00,4/1/2014,15:00:00,4/1/2014,15:05:00,4/1/2014,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,STREET,40.825000,-73.916615,"(40.825000207, -73.916615377)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
2014-04-06 14:45:00,4/6/2014,14:45:00,4/6/2014,14:45:00,4/6/2014,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,STREET,40.830013,-73.918112,"(40.830013132, -73.918112312)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
2014-05-06 14:00:00,5/6/2014,14:00:00,5/6/2014,14:05:00,5/6/2014,GRAND LARCENY OF MOTOR VEHICLE,"LARCENY,GRAND OF AUTO",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,STREET,40.828608,-73.921742,"(40.828607613, -73.92174192)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
2014-03-06 23:42:00,3/6/2014,23:42:00,3/6/2014,23:42:00,3/7/2014,POSSESSION OF STOLEN PROPERTY,"STOLEN PROPERTY 2,1,POSSESSION",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,STREET,40.838752,-73.913758,"(40.83875187, -73.913757556)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...


In [7]:
df_data = dfCopy.copy()
df_data['Time'] = df_data.index.hour

In [9]:
df_data['Morning'] = df_data['Time'].between(06.01,12.0)
df_data['Afternoon'] = df_data['Time'].between(12.01,17.0)
df_data['Evening'] = df_data['Time'].between(17.01,20.0)
df_data['Night'] = df_data['Time'].between(20.01,06.0)

In [46]:
data = df_data.drop(['CMPLNT_FR_DT','CMPLNT_FR_TM','CMPLNT_TO_DT','CMPLNT_TO_TM','RPT_DT','ADDR_PCT_CD','Latitude','Longitude','Lat_Lon','Shape_Area','Shape_Leng','the_geom'],axis=1)

In [54]:
filter_data = filterData(data,'Precinct',44)

In [55]:
temp = filter_data['OFNS_DESC'].value_counts().head(5).reset_index()
temp

,index,OFNS_DESC
0,DANGEROUS DRUGS,21362
1,ASSAULT 3 & RELATED OFFENSES,17468
2,PETIT LARCENY,16726
3,HARRASSMENT 2,15399
4,CRIMINAL MISCHIEF & RELATED OF,11569


In [56]:
final_data = filter_data.loc[filter_data['OFNS_DESC'].isin(temp['index'])]

In [57]:
final_data['OFNS_DESC'].value_counts()

DANGEROUS DRUGS                   21362
ASSAULT 3 & RELATED OFFENSES      17468
PETIT LARCENY                     16726
HARRASSMENT 2                     15399
CRIMINAL MISCHIEF & RELATED OF    11569
Name: OFNS_DESC, dtype: int64

In [61]:
final_data.shape

(82524, 13)

In [62]:
def getFinalData():
    return final_data